In [0]:
!pip install --upgrade face_recognition
!pip install --upgrade opencv-python

In [0]:
import face_recognition
import cv2
import numpy

from IPython.display import display, Javascript, Image, clear_output
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import urllib

!wget -O me.jpg 'https://media.licdn.com/dms/image/C4D03AQHDsuwJf6rX0w/profile-displayphoto-shrink_200_200/0?e=1574899200&v=beta&t=Ny70D9QQIWbtkZGngLPn2yPih10tf0W-m6qwHM3NtXQ'
!ls -la

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  resp = urllib.request.urlopen(data)
  image = numpy.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  return image

eu_image = face_recognition.load_image_file("me.jpg")
eu_face_encoding = face_recognition.face_encodings(eu_image)[0]

known_face_encodings = [
    eu_face_encoding,
]
known_face_names = [
    "Matheus",
]

In [0]:
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
first = True

filename = 'result.jpg'
scale = 4

while True:
    frame = take_photo()
    small_frame = cv2.resize(frame, (0, 0), fx=1/scale, fy=1/scale)
    rgb_small_frame = small_frame[:, :, ::-1]
    
    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Homossexual"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = numpy.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= scale
        right *= scale
        bottom *= scale
        left *= scale

        cv2.rectangle(frame, (left, top), (right, bottom), (100, 0, 100), 2)

        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (100, 0, 100), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
    clear_output()
    cv2_imshow(frame)
